# {meth}`~matplotlib.axes.Axes.imshow` 之 `origin` 和 `extent`

{meth}`~matplotlib.axes.Axes.imshow` 允许将图像（要么是二维数组，它将根据 `norm` 和 `cmap` 进行着色处理，要么是三维的 RGB(A) 数组，它将被直接使用）渲染到数据空间中的矩形区域。最终渲染中图像的方向由 ``origin`` 和 ``extent`` 关键字参数（以及在结果 {class}`~matplotlib.image.AxesImage` 实例上的属性）以及坐标轴的数据限制来控制。

`extent` 关键字参数控制图像将填充的数据坐标系中的边界框，指定为 ``(left, right, bottom, top)`` 数据坐标。`origin` 关键字参数控制图像如何填充该边界框，并且最终渲染图像中的方向也受坐标轴的限制影响。

```{hint}
没问题，下面是您需要的翻译：

下面的大部分代码用于向绘图中添加标签和说明性文本。*origin* 和 *extent* 的效果可以在绘图中看到，而无需关注所有代码细节。为了快速理解，您可能希望跳过下面的代码细节，并直接继续讨论结果。
```

In [1]:
# %matplotlib tk
import matplotlib.pyplot as plt
import numpy as np

from matplotlib.gridspec import GridSpec


def index_to_coordinate(index, extent, origin):
    """Return the pixel center of an index."""
    left, right, bottom, top = extent

    hshift = 0.5 * np.sign(right - left)
    left, right = left + hshift, right - hshift
    vshift = 0.5 * np.sign(top - bottom)
    bottom, top = bottom + vshift, top - vshift

    if origin == 'upper':
        bottom, top = top, bottom

    return {
        "[0, 0]": (left, bottom),
        "[M', 0]": (left, top),
        "[0, N']": (right, bottom),
        "[M', N']": (right, top),
    }[index]


def get_index_label_pos(index, extent, origin, inverted_xindex):
    """
    Return the desired position and horizontal alignment of an index label.
    """
    if extent is None:
        extent = lookup_extent(origin)
    left, right, bottom, top = extent
    x, y = index_to_coordinate(index, extent, origin)

    is_x0 = index[-2:] == "0]"
    halign = 'left' if is_x0 ^ inverted_xindex else 'right'
    hshift = 0.5 * np.sign(left - right)
    x += hshift * (1 if is_x0 else -1)
    return x, y, halign


def get_color(index, data, cmap):
    """Return the data color of an index."""
    val = {
        "[0, 0]": data[0, 0],
        "[0, N']": data[0, -1],
        "[M', 0]": data[-1, 0],
        "[M', N']": data[-1, -1],
    }[index]
    return cmap(val / data.max())


def lookup_extent(origin):
    """Return extent for label positioning when not given explicitly."""
    if origin == 'lower':
        return (-0.5, 6.5, -0.5, 5.5)
    else:
        return (-0.5, 6.5, 5.5, -0.5)


def set_extent_None_text(ax):
    ax.text(3, 2.5, 'equals\nextent=None', size='large',
            ha='center', va='center', color='w')


def plot_imshow_with_labels(ax, data, extent, origin, xlim, ylim):
    """Actually run ``imshow()`` and add extent and index labels."""
    im = ax.imshow(data, origin=origin, extent=extent)

    # extent labels (left, right, bottom, top)
    left, right, bottom, top = im.get_extent()
    if xlim is None or top > bottom:
        upper_string, lower_string = 'top', 'bottom'
    else:
        upper_string, lower_string = 'bottom', 'top'
    if ylim is None or left < right:
        port_string, starboard_string = 'left', 'right'
        inverted_xindex = False
    else:
        port_string, starboard_string = 'right', 'left'
        inverted_xindex = True
    bbox_kwargs = {'fc': 'w', 'alpha': .75, 'boxstyle': "round4"}
    ann_kwargs = {'xycoords': 'axes fraction',
                  'textcoords': 'offset points',
                  'bbox': bbox_kwargs}
    ax.annotate(upper_string, xy=(.5, 1), xytext=(0, -1),
                ha='center', va='top', **ann_kwargs)
    ax.annotate(lower_string, xy=(.5, 0), xytext=(0, 1),
                ha='center', va='bottom', **ann_kwargs)
    ax.annotate(port_string, xy=(0, .5), xytext=(1, 0),
                ha='left', va='center', rotation=90,
                **ann_kwargs)
    ax.annotate(starboard_string, xy=(1, .5), xytext=(-1, 0),
                ha='right', va='center', rotation=-90,
                **ann_kwargs)
    ax.set_title(f'origin: {origin}')

    # index labels
    for index in ["[0, 0]", "[0, N']", "[M', 0]", "[M', N']"]:
        tx, ty, halign = get_index_label_pos(index, extent, origin,
                                             inverted_xindex)
        facecolor = get_color(index, data, im.get_cmap())
        ax.text(tx, ty, index, color='white', ha=halign, va='center',
                bbox={'boxstyle': 'square', 'facecolor': facecolor})
    if xlim:
        ax.set_xlim(*xlim)
    if ylim:
        ax.set_ylim(*ylim)


def generate_imshow_demo_grid(extents, xlim=None, ylim=None):
    N = len(extents)
    fig = plt.figure(tight_layout=True)
    fig.set_size_inches(6, N * (11.25) / 5)
    gs = GridSpec(N, 5, figure=fig)

    columns = {'label': [fig.add_subplot(gs[j, 0]) for j in range(N)],
               'upper': [fig.add_subplot(gs[j, 1:3]) for j in range(N)],
               'lower': [fig.add_subplot(gs[j, 3:5]) for j in range(N)]}
    x, y = np.ogrid[0:6, 0:7]
    data = x + y

    for origin in ['upper', 'lower']:
        for ax, extent in zip(columns[origin], extents):
            plot_imshow_with_labels(ax, data, extent, origin, xlim, ylim)

    columns['label'][0].set_title('extent=')
    for ax, extent in zip(columns['label'], extents):
        if extent is None:
            text = 'None'
        else:
            left, right, bottom, top = extent
            text = (f'left: {left:0.1f}\nright: {right:0.1f}\n'
                    f'bottom: {bottom:0.1f}\ntop: {top:0.1f}\n')
        ax.text(1., .5, text, transform=ax.transAxes, ha='right', va='center')
        ax.axis('off')
    return columns

## 默认 `extent`

默认的 `extent=None`：

In [2]:
generate_imshow_demo_grid(extents=[None])

{'label': [<Axes: title={'center': 'extent='}>],
 'upper': [<Axes: title={'center': 'origin: upper'}>],
 'lower': [<Axes: title={'center': 'origin: lower'}>]}

通常，对于形状为 `(M, N)` 的数组，第一个索引沿垂直方向运行，第二个索引沿水平方向运行。像素中心位于从 `0` 到 ``N' = N - 1`` 的水平位置和从 `0` 到 ``M' = M - 1`` 的垂直位置。*origin* 决定了如何在边界框中填充数据。

对于 ``origin='lower'``：

- [0, 0] 在左下角
- [M', 0] 在左上角
- [0, N'] 在右下角
- [M', N'] 在右上角

``origin='upper'`` 反转了垂直轴的方向和填充：

- [0, 0] 在左上角
- [M', 0] 在左下角
- [0, N'] 在右上角
- [M', N'] 在右下角

总之，[0, 0] 索引的位置以及范围都受到 *origin* 的影响：

```{eval-rst}
======  ===============  ==========================================
origin  [0, 0] position  extent
======  ===============  ==========================================
upper   top left         ``(-0.5, numcols-0.5, numrows-0.5, -0.5)``
lower   bottom left      ``(-0.5, numcols-0.5, -0.5, numrows-0.5)``
======  ===============  ==========================================
```

*origin* 的默认值由 {rc}`image.origin` 设置，默认为 ``'upper'``，以匹配数学和计算机图形图像索引约定中的矩阵索引约定。

## 显式范围

通过设置 *extent*，我们可以定义图像区域的坐标。底层图像数据将被插值/重新采样以填充该区域。

如果 Axes 设置为自动缩放，那么 Axes 的视图限制将设置为与 *extent* 匹配，以确保由 ``(left, bottom)`` 设置的坐标位于 Axes 的左下角！然而，这可能会反转轴，使它们不按“自然”方向增加。

In [3]:
extents = [(-0.5, 6.5, -0.5, 5.5),
           (-0.5, 6.5, 5.5, -0.5),
           (6.5, -0.5, -0.5, 5.5),
           (6.5, -0.5, 5.5, -0.5)]

columns = generate_imshow_demo_grid(extents)
set_extent_None_text(columns['upper'][1])
set_extent_None_text(columns['lower'][0])

## 显式范围和轴限制

如果我们通过显式设置 {meth}`~matplotlib.axes.Axes.set_xlim` / {meth}`~matplotlib.axes.Axes.set_ylim` 来固定轴的限制，我们强制了轴的特定大小和方向。这可能会使图像的“左右”和“上下”感觉与屏幕上的方向解耦。

在下面的示例中，我们选择的限制略大于范围（注意轴内的白色区域）。

虽然我们保持与之前示例相同的范围，但坐标 (0, 0) 现在明确地放在左下角，并且值从左到右、自上而下增加（从观察者的角度看）。我们可以看到：

- 坐标 ``(left, bottom)`` 锚定了图像，然后图像填充了盒子，朝向数据空间中的 ``(right, top)`` 点。
- 第一列总是最靠近“左”。
- *origin* 控制第一行是靠近“上”还是“下”。
- 图像可能在任一方向上被翻转。
- 图像的“左右”和“上下”感觉可能与屏幕上的方向解耦。

In [4]:
generate_imshow_demo_grid(extents=[None] + extents,
                          xlim=(-2, 8), ylim=(-1, 6))

plt.show()